In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import csv
import numpy as np

DATA_DIR = 'MNIST'
sentiment_data = 'sentiment-data'


def getWordVectorDict():
    reader = csv.reader(open(sentiment_data +'/word-vectors-refine.txt'))

    word_vector_dict = {}
    for row in reader:
        key = row[0]
        if key in word_vector_dict:
            # implement your duplicate row handling here
            pass
        word_vector_dict[key] = np.array(row[1:])
    return word_vector_dict

def getPaddedSentenceMatrix(sentenceMatrix):
    wordCount = 100
    return np.vstack((sentenceMatrix, np.zeros((wordCount - np.shape(sentenceMatrix)[0],np.shape(sentenceMatrix)[1]), dtype=np.float32)))

def getVectorForSentence(sentence, word_vec_dict):
    sentence_matrix = []
    for word in sentence.split(' '):
        word_vec = word_vec_dict[word]
        if(len(sentence_matrix) == 0):
            sentence_matrix = word_vec
        else:
            sentence_matrix = np.vstack((sentence_matrix,word_vec))
    return getPaddedSentenceMatrix(sentence_matrix)

def getData(fileName):
    reader = csv.reader(open(sentiment_data +'/' + fileName))
    trainingData = []
    for row in reader:
        data = {}
        data['label'] =  1 if row[0] == 'postive' else 0
        data['sentence'] = row[1:]
        trainingData.append(data)
    return trainingData

word_vec_dict = getWordVectorDict()


def transform(row):
    return row['label'], getVectorForSentence(row['sentence'][0], word_vec_dict)


word_vector_size = 50;
time_steps = 100;
num_classes = 2
batch_size = 1000;
n_iterations = 10;
hidden_layer_size = 50

training_data = getData('train.csv')
training_rows  = map(lambda row: transform(row), training_data)
training_data = map(lambda row: row[1], training_rows)
training_labels = map(lambda row: row[0], training_rows)
#test_data = map(lambda row: transform(row), getData('test.csv'))
test_data = getData('test.csv')
test_rows  = map(lambda row: transform(row), test_data)
test_data = map(lambda row: row[1], test_rows)
test_labels = map(lambda row: row[0], test_rows)
print training_data[0]

[['0.68491' '0.32385' '-0.11592' ..., '0.17874' '-0.1693' '0.062375']
 ['0.96193' '0.012516' '0.21733' ..., '0.14032' '-0.38468' '-0.38712']
 ['0.6008' '0.18044' '0.078339' ..., '-0.016404' '-0.65372' '-0.38255']
 ..., 
 ['0.88387' '-0.14199' '0.13566' ..., '0.52711' '-0.20148' '0.0095952']
 ['-0.0010919' '0.33324' '0.35743' ..., '-0.45697' '-0.048969' '1.1316']
 ['-0.55114' '-0.16296' '-0.95494' ..., '-1.0346' '-0.25143' '1.4836']]


In [2]:

#Setting up the input and labels placeholders
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,
                                            word_vector_size])
y = tf.placeholder(tf.int32, shape=[None, num_classes])
y_one_hot = tf.one_hot( y , num_classes )

# TensorFlow built-in functions
# Creating the RNN cell and creating the outputs
with tf.variable_scope("gru"):
    gru_cell = tf.contrib.rnn.GRUCell(hidden_layer_size)
    outputs, states = tf.nn.dynamic_rnn(gru_cell,_inputs, dtype=tf.float32)

weights = {
        'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size,
                                                         num_classes],
                                                         mean=0,stddev=.01))
}


# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states,
                         weights["linear_layer"])
softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,
                                                  labels = y)                         
cross_entropy = tf.reduce_mean(softmax)

train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1),
                              tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100






In [3]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

label_batch = np.array(training_labels[100 : 200])
l = np.array(dense_to_one_hot(label_batch,2)).reshape(len(label_batch), num_classes)

print l.shape

(100, 2)


In [4]:
batch_size = 100;
n_iterations = 20;

#Initialize session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #Initialize test data
    #test_data = mnist.test.images[:batch_size].reshape((-1,time_steps, word_vector_size))
    #test_label = mnist.test.labels[:batch_size]

    for epoch in range(n_iterations):
        print "Epoch:", epoch
        for j in range(len(training_data)/batch_size):
            #print "j:", j
            if(j == 100):
                break
            startIndex = j*batch_size
            endIndex = startIndex + batch_size
            batch_x = np.array(training_data[startIndex : endIndex]).reshape((-1,time_steps, word_vector_size))
            #print len(training_labels[startIndex : endIndex])
            batch_y = dense_to_one_hot(np.array(training_labels[startIndex : endIndex]),num_classes)
            #print "X:", len(batch_x)
            #print "Y:", len(batch_y)
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_x = batch_x.reshape((batch_size, time_steps, word_vector_size))
            sess.run(train_step, feed_dict={_inputs: batch_x,
                                            y: batch_y})
            if epoch % 1 == 0:
                acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
                loss = sess.run(cross_entropy, feed_dict={_inputs: batch_x, y: batch_y})
                print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                       "{:.6f}".format(loss) + ", Training Accuracy= " + \
                       "{:.5f}".format(acc))


    print ("Testing Accuracy:",
        sess.run(accuracy, feed_dict={_inputs: test_data, y: dense_to_one_hot(np.array(test_labels), num_classes)}))

Epoch: 0
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692626, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.691962, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692582, Training Accuracy= 56.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692313, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693548, Training Accuracy= 46.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693797, Training Accuracy= 41.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.694005, Training Accuracy= 42.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693136, Training Accuracy= 48.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693059, Training Accuracy= 48.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693771, Training Accuracy= 50.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692705, Training Accuracy= 57.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.694348, Training Accuracy= 43.00000
100
X: 100
Y: 100
Iter 0, Minib

Iter 1, Minibatch Loss= 0.689533, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.684541, Training Accuracy= 57.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.693339, Training Accuracy= 43.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.686362, Training Accuracy= 57.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.695805, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.687973, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690148, Training Accuracy= 58.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.688296, Training Accuracy= 60.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.687993, Training Accuracy= 59.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.687677, Training Accuracy= 62.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.685917, Training Accuracy= 59.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.681711, Training Accuracy= 60.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.682301, Traini

Iter 2, Minibatch Loss= 0.546955, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.618658, Training Accuracy= 65.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.607287, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.621255, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.588148, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.553629, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.601077, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.560825, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.563519, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.517260, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.584381, Training Accuracy= 71.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.576109, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.614754, Traini

Y: 100
Iter 3, Minibatch Loss= 0.582413, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.526545, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.519334, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.531055, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.509268, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.519197, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.460636, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.554439, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.521184, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.569488, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.539255, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.612448, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.540365,

Iter 4, Minibatch Loss= 0.494463, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.512331, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.483388, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.445317, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.529774, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.495025, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.539498, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.514500, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.598059, Training Accuracy= 69.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.532146, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.494456, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.430802, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.556724, Traini

...

Iter 15, Minibatch Loss= 0.439203, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.295139, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.401235, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.389958, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.316801, Training Accuracy= 89.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.302471, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.273687, Training Accuracy= 90.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.274332, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.411982, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.372227, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.315146, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.437863, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.3

Iter 16, Minibatch Loss= 0.393863, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.384452, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.309368, Training Accuracy= 90.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.288956, Training Accuracy= 86.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.264594, Training Accuracy= 91.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.265710, Training Accuracy= 89.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.403599, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.363782, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.302606, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.427313, Training Accuracy= 80.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.355181, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.358751, Training Accuracy= 86.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.3

Y: 100
Iter 17, Minibatch Loss= 0.301407, Training Accuracy= 90.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.275924, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.255802, Training Accuracy= 92.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.256902, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.393632, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.355139, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.289810, Training Accuracy= 89.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.416136, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.347968, Training Accuracy= 86.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.348795, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.373121, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.325805, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 17, Minibatch Lo

Iter 18, Minibatch Loss= 0.262331, Training Accuracy= 89.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.246000, Training Accuracy= 92.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.247907, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.385319, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.347606, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.276269, Training Accuracy= 89.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.405776, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.342135, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.338174, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.366736, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.320652, Training Accuracy= 86.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.204675, Training Accuracy= 93.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.2

Iter 19, Minibatch Loss= 0.252621, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.376619, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.347273, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.263524, Training Accuracy= 91.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.416152, Training Accuracy= 80.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.341204, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.332111, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.360060, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.313265, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.197700, Training Accuracy= 92.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.231494, Training Accuracy= 90.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.459074, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.3